In [3]:
from public_data.models import Ocsge, Commune, CouvertureSol
from django.db import connection
city = Commune.objects.get(name="Auch")
bbox = city.mpoly.extent
city, bbox

(<Commune: Auch (32013)>,
 (0.486067241998179, 43.6175987830157, 0.647056360983016, 43.7076959675878))

In [9]:
query = f"""
SELECT
    ST_NPoints(mpoly) AS geometry,
    ST_NPoints(ST_SimplifyPreserveTopology(mpoly, 0.000000001)),
    ST_NPoints(ST_SimplifyPreserveTopology(mpoly, 0.00000001)),
    ST_NPoints(ST_SimplifyPreserveTopology(mpoly, 0.0000001)),
    ST_NPoints(ST_SimplifyPreserveTopology(mpoly, 0.000001)),
    ST_NPoints(ST_SimplifyPreserveTopology(mpoly, 0.00001)),
    ST_NPoints(ST_SimplifyPreserveTopology(mpoly, 0.0001)),
    ST_NPoints(ST_SimplifyPreserveTopology(mpoly, 0.001)),
    ST_NPoints(ST_SimplifyPreserveTopology(mpoly, 0.01))
FROM {Ocsge._meta.db_table} as o INNER JOIN {CouvertureSol._meta.db_table} as t ON t.code_prefix = o.couverture
WHERE mpoly && ST_MakeEnvelope(%s, %s, %s, %s, 4326)
ORDER BY RANDOM() DESC
LIMIT 10
"""
with connection.cursor() as cursor:
    cursor.execute(query, bbox)
    rows = cursor.fetchall()
rows

[(18, 17, 15, 10, 10, 10, 5, 5, 5),
 (15, 15, 15, 12, 10, 9, 8, 5, 5),
 (25, 25, 23, 17, 15, 15, 5, 5, 5),
 (31, 31, 31, 26, 24, 17, 7, 5, 5),
 (19, 19, 19, 11, 10, 10, 5, 5, 5),
 (57, 56, 54, 28, 25, 19, 8, 5, 5),
 (16, 16, 14, 12, 6, 6, 5, 5, 5),
 (14, 14, 13, 9, 8, 8, 5, 5, 5),
 (13, 13, 12, 6, 6, 6, 5, 5, 5),
 (75, 75, 73, 51, 36, 24, 9, 5, 5)]